lanjut dari part 1, sedikit repetisi tentang clean

# Data
application_train/application_test: data pelatihan dan pengujian utama dengan informasi tentang setiap aplikasi pinjaman di Home Credit. Setiap pinjaman memiliki barisnya masing-masing dan ditandai dengan fitur SK_ID_CURR. Data aplikasi pelatihan dilengkapi dengan TARGET yang menunjukkan 0: pinjaman telah dilunasi atau 1: pinjaman tidak dilunasi.

bureau: data mengenai kredit klien sebelumnya dari lembaga keuangan lain. Setiap kredit sebelumnya memiliki barisnya sendiri di biro, tetapi satu pinjaman dalam data aplikasi dapat memiliki beberapa kredit sebelumnya.

bureau_balance: data bulanan tentang kredit sebelumnya di biro. Setiap baris adalah satu bulan dari kredit sebelumnya, dan satu kredit sebelumnya dapat memiliki beberapa baris, satu untuk setiap bulan dari panjang kredit.

previous_application: aplikasi sebelumnya untuk pinjaman di Home Credit klien yang memiliki pinjaman dalam data aplikasi. Setiap pinjaman saat ini dalam data aplikasi dapat memiliki beberapa pinjaman sebelumnya. Setiap aplikasi sebelumnya memiliki satu baris dan diidentifikasi oleh fitur SK_ID_PREV.

POS_CASH_BALANCE: data bulanan tentang titik penjualan sebelumnya atau pinjaman tunai yang dimiliki klien dengan Home Credit. Setiap baris adalah satu bulan dari titik penjualan atau pinjaman tunai sebelumnya, dan satu pinjaman sebelumnya dapat memiliki banyak baris.

credit_card_balance: data bulanan tentang kartu kredit sebelumnya yang dimiliki klien dengan Home Credit. Setiap baris adalah satu bulan dari saldo kartu kredit, dan satu kartu kredit dapat memiliki banyak baris.

installments_payment: riwayat pembayaran untuk pinjaman sebelumnya di Home Credit. Ada satu baris untuk setiap pembayaran yang dilakukan dan satu baris untuk setiap pembayaran yang terlewatkan.



![image](https://drive.google.com/uc?id=1uMUlwp_cnYv8sIfGPgiXf-uxBxAzptqM)


In [1]:
#import libraries
import pandas as pd
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
train_dat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_dat.csv')
test_dat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_dat.csv')

# Feature Engineering


## Data Biro

In [4]:
bureau = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bureau.csv')

In [5]:
# Menggabungkan fitur numerik
grp = bureau.drop(['SK_ID_BUREAU'], axis = 1).groupby(by=['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
application_bureau = train_dat.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = test_dat.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))
# Menggabungkan fitur kategorikal
bureau_categorical = pd.get_dummies(bureau.select_dtypes('object'))
bureau_categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
grp = bureau_categorical.groupby(by = ['SK_ID_CURR']).mean().reset_index()
grp.columns = ['BUREAU_'+column if column !='SK_ID_CURR' else column for column in grp.columns]
application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau.update(application_bureau[grp.columns].fillna(0))

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test.update(application_bureau_test[grp.columns].fillna(0))

In [6]:
# Jumlah pinjaman masa lalu per pelanggan
grp = bureau.groupby(by = ['SK_ID_CURR'])['SK_ID_BUREAU'].count().reset_index().rename(columns = {'SK_ID_BUREAU': 'BUREAU_LOAN_COUNT'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_COUNT'] = application_bureau['BUREAU_LOAN_COUNT'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_COUNT'] = application_bureau_test['BUREAU_LOAN_COUNT'].fillna(0)
# Jumlah jenis pinjaman masa lalu per pelanggan
grp = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

application_bureau = application_bureau.merge(grp, on='SK_ID_CURR', how='left')
application_bureau['BUREAU_LOAN_TYPES'] = application_bureau['BUREAU_LOAN_TYPES'].fillna(0)

application_bureau_test = application_bureau_test.merge(grp, on='SK_ID_CURR', how='left')
application_bureau_test['BUREAU_LOAN_TYPES'] = application_bureau_test['BUREAU_LOAN_TYPES'].fillna(0)


In [7]:

# Utang atas rasio kredit
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM': 'TOTAL_CREDIT_SUM'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CREDIT_SUM_DEBT'})

grp1['DEBT_CREDIT_RATIO'] = grp2['TOTAL_CREDIT_SUM_DEBT']/grp1['TOTAL_CREDIT_SUM']

del grp1['TOTAL_CREDIT_SUM']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['DEBT_CREDIT_RATIO'] = application_bureau['DEBT_CREDIT_RATIO'].fillna(0)

# Replace inf and -inf values with 0 in 'DEBT_CREDIT_RATIO' column
application_bureau['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0, inplace=True)
application_bureau['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau['DEBT_CREDIT_RATIO'], downcast='float')


application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['DEBT_CREDIT_RATIO'] = application_bureau_test['DEBT_CREDIT_RATIO'].fillna(0)

# Replace inf and -inf values with 0 in 'DEBT_CREDIT_RATIO' column in test data
application_bureau_test['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], 0, inplace=True)
application_bureau_test['DEBT_CREDIT_RATIO'] = pd.to_numeric(application_bureau_test['DEBT_CREDIT_RATIO'], downcast='float')

In [8]:
(application_bureau[application_bureau['DEBT_CREDIT_RATIO'] > 0.5]['TARGET'].value_counts()/len(application_bureau[application_bureau['DEBT_CREDIT_RATIO'] > 0.5]))*100

0    88.385852
1    11.614148
Name: TARGET, dtype: float64

In [9]:
# Jatuh tempo atas rasio utang
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)

grp1 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

grp2 = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(by=['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename(columns={'AMT_CREDIT_SUM_DEBT':'TOTAL_CUSTOMER_DEBT'})

grp1['OVERDUE_DEBT_RATIO'] = grp1['TOTAL_CUSTOMER_OVERDUE']/grp2['TOTAL_CUSTOMER_DEBT']

del grp1['TOTAL_CUSTOMER_OVERDUE']

application_bureau = application_bureau.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau['OVERDUE_DEBT_RATIO'] = application_bureau['OVERDUE_DEBT_RATIO'].fillna(0)
# Replace inf and -inf values with 0 in 'OVERDUE_DEBT_RATIO' column
application_bureau['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0, inplace=True)
application_bureau['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau['OVERDUE_DEBT_RATIO'], downcast='float')

application_bureau_test = application_bureau_test.merge(grp1, on='SK_ID_CURR', how='left')
application_bureau_test['OVERDUE_DEBT_RATIO'] = application_bureau_test['OVERDUE_DEBT_RATIO'].fillna(0)

# Replace inf and -inf values with 0 in 'DEBT_CREDIT_RATIO' column in test data
application_bureau_test['OVERDUE_DEBT_RATIO'].replace([np.inf, -np.inf], 0, inplace=True)
application_bureau_test['OVERDUE_DEBT_RATIO'] = pd.to_numeric(application_bureau_test['OVERDUE_DEBT_RATIO'], downcast='float')

In [10]:
import gc

gc.collect()

0

In [13]:
(application_bureau[application_bureau['DEBT_CREDIT_RATIO'] > 0.5]['TARGET'].value_counts()/len(application_bureau[application_bureau['DEBT_CREDIT_RATIO'] > 0.5]))*100

0    88.385852
1    11.614148
Name: TARGET, dtype: float64

## Data Aplikasi Sebelumnya

In [14]:
previous_application = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/previous_application.csv')

In [15]:
def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    return first + second == 2

isOneToOne(previous_application,'SK_ID_CURR','SK_ID_PREV')

False

fungsi bernama isOneToOne yang menerima tiga argumen:

1. df: Sebuah pandas DataFrame, yang mewakili data masukan.
2. col1: Sebuah string, yang mewakili nama kolom pertama yang akan dibandingkan.
3. col2: Sebuah string, yang mewakili nama kolom kedua yang akan dibandingkan.

Fungsi tersebut memeriksa apakah hubungan antara nilai-nilai dalam kolom col1 dan col2 adalah satu-ke-satu (one-to-one) dalam DataFrame df yang diberikan. Hal ini dilakukan dengan melakukan langkah-langkah berikut:

1. Menghapus baris duplikat berdasarkan kolom col1 dan col2 menggunakan metode drop_duplicates().
2. Mengelompokkan DataFrame hasilnya berdasarkan kolom col1 dan menghitung nilai unik dalam kolom col2 menggunakan metode groupby() dan count().
3. Mengambil nilai maksimum dari jumlah nilai unik dalam kolom col2 untuk setiap kelompok nilai dalam kolom col1, dan menyimpannya dalam variabel first.
4. Mengelompokkan DataFrame hasilnya berdasarkan kolom col2 dan menghitung nilai unik dalam kolom col1 menggunakan metode groupby() dan count().
5. Mengambil nilai maksimum dari jumlah nilai unik dalam kolom col1 untuk setiap kelompok nilai dalam kolom col2, dan menyimpannya dalam variabel second.
6. Mengembalikan True jika jumlah dari first dan second sama dengan 2, yang menunjukkan bahwa hubungan antara col1 dan col2 adalah satu-ke-satu (yaitu setiap nilai unik dalam col1 dipetakan ke tepat satu nilai unik dalam col2 dan sebaliknya), jika tidak mengembalikan False.

Fungsi tersebut kemudian dipanggil dengan DataFrame previous_application, dan nama kolom `SK_ID_CURR` dan `SK_ID_PREV` sebagai argumen, untuk memeriksa apakah hubungan antara kedua kolom ini adalah satu-ke-satu dalam DataFrame yang diberikan.

In [16]:
# Jumlah aplikasi sebelumnya per pelanggan
grp = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index().rename(columns={'SK_ID_PREV':'PREV_APP_COUNT'})

# Ambil hanya ID yang ada di application_bureau
application_bureau_prev = application_bureau.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test = application_bureau_test.merge(grp, on =['SK_ID_CURR'], how = 'left')

#Isi NA untuk hitungan aplikasi sebelumnya (misalkan ada ID aplikasi yang ada di biro aplikasi tetapi tidak ada
# dalam grp, berarti orang tersebut tidak pernah mengambil pinjaman sebelumnya, 
#jadi jumlah pinjaman sebelumnya untuk orang tersebut = 0)
application_bureau_prev['PREV_APP_COUNT'] = application_bureau_prev['PREV_APP_COUNT'].fillna(0)
application_bureau_prev_test['PREV_APP_COUNT'] = application_bureau_prev_test['PREV_APP_COUNT'].fillna(0)

In [17]:
# Menggabungkan fitur numerik

#Ambil mean dari semua parameter (pengelompokan berdasarkan SK_ID_CURR)
grp = previous_application.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()

#Tambahkan awalan prev di depan semua kolom agar kita tahu bahwa kolom tersebut berasal dari aplikasi_sebelumnya
prev_columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]

#ganti kolomnya
grp.columns = prev_columns

application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [18]:
# Menggabungkan fitur kategorikal
prev_categorical = pd.get_dummies(previous_application.select_dtypes('object'))
prev_categorical['SK_ID_CURR'] = previous_application['SK_ID_CURR']

grp = prev_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['PREV_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]

application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

In [19]:
gc.collect()

0

## Data POS_CASH_balance

In [20]:
pos_cash= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/POS_CASH_balance.csv')

In [21]:
# Menggabungkan fitur numerik
grp = pos_cash.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))

# Menggabungkan fitur kategorikal
pos_cash_categorical = pd.get_dummies(pos_cash.select_dtypes('object'))
pos_cash_categorical['SK_ID_CURR'] = pos_cash['SK_ID_CURR']

grp = pos_cash_categorical.groupby('SK_ID_CURR').mean().reset_index()
grp.columns = ['POS_'+column if column != 'SK_ID_CURR' else column for column in grp.columns]
application_bureau_prev = application_bureau_prev.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))

application_bureau_prev_test = application_bureau_prev_test.merge(grp, on=['SK_ID_CURR'], how='left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))
gc.collect()

0

## Data installments_payments

In [22]:
insta_payments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/installments_payments.csv')

In [23]:
# Menggabungkan fitur numerik dan tidak ada fitur kategorikal dalam dataset ini
grp = insta_payments.drop('SK_ID_PREV', axis =1).groupby(by=['SK_ID_CURR']).mean().reset_index()
prev_columns = ['INSTA_'+column if column != 'SK_ID_CURR' else column for column in grp.columns ]
grp.columns = prev_columns
application_bureau_prev = application_bureau_prev.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev.update(application_bureau_prev[grp.columns].fillna(0))
application_bureau_prev_test = application_bureau_prev_test.merge(grp, on =['SK_ID_CURR'], how = 'left')
application_bureau_prev_test.update(application_bureau_prev_test[grp.columns].fillna(0))
gc.collect()

0